In [8]:
import torch
import tqdm as notebook_tq
import codecs

In [10]:
!pwd

/nas/home/skadvani/ProtoQA_GPT2


In [9]:
import os
os.chdir('ProtoQA_GPT2')

FileNotFoundError: [Errno 2] No such file or directory: 'ProtoQA_GPT2'

In [14]:
import json 
import pandas as pd  

# test_file_path = 'ProtoQA_GPT2/data/dev/dev.crowdsourced.jsonl'
# predicted_path = 'ProtoQA_GPT2/sample_answer.json'
correct_label_file_path =  'data/train/train.jsonl'
predicted_path =  'final_outputsample_answers.json'
predicted_df = pd.read_json(predicted_path,lines=True)

correct_label_df = pd.read_json(correct_label_file_path,lines=True)

In [12]:
import gensim.downloader as api
import string


wv = api.load('word2vec-google-news-300')

In [20]:
'''
This function checks similarity of word1, word2 have multiple words in it 
e.g. 
'''
import re
def removePunctuation(word):
    return re.sub(r"[,.;@#?!/&$]+\ *", " ", word)
def checkPairwise(word1,word2):
    temp = 0.0
    word1, word2= removePunctuation(word1), removePunctuation(word2)
    words1 = word1.split(" ")
    words2 = word2.split(" ")
    #print(words1, " fd ",  words2)
    try:
        for word1 in words1:
            sim = []
            for word2 in words2:
                temp += wv.similarity(word1,word2)
    except:
        pass
    if ((len(word1) + len(word2)) == 0):
        return 0
    return temp / (len(word1) + len(word2))

#Function checks gensim similarity between single words and list of correct keys 
# e.g. : (bottle, ['key', 'chain', 'water bottle'] like that   
def checkSimilarity(candidateList, key):
    maxx = 0
    for word in candidateList:
        maxx = max(checkPairwise(word,key), maxx)
    if maxx > 0.2:return True
    return False

In [26]:
correct_token = 0
false_token = 0
label = 0
generate_dataset=[]
text_data=[]
label_data=[]
correct = open('correct.txt','w')
incorrect = open('incorrect.txt','w')
for index in range(8782):
    question_id = correct_label_df['metadata'][index]['id']
    question = correct_label_df['question'][index]['original']
    for token in list(predicted_df[question_id][0].keys()):
        if checkSimilarity(list((correct_label_df['answers'][index]['raw']).keys()),token):
        #if token in list((correct_label_df['answers'][index]['raw']).keys()):
            label = 0
            correct.write((" ".join(list((correct_label_df['answers'][index]['raw']).keys())) + " " +token))
            correct_token+=1
        else:
            label = 1
            #print("'"+token+"'", (correct_label_df['answers'][index]['raw']).keys())
            incorrect.write(" ".join(list((correct_label_df['answers'][index]['raw']).keys()))+ " " +token)
            false_token+=1
        temp={}
        label_data.append(str(label))
        temp['label'] = label
        temp['text'] = question + " " + token
        text_data.append(question + " " + token)
        
        generate_dataset.append(temp)
print('Correct Token: ', correct_token)
print('False Token: ', false_token)
total_token = correct_token + false_token
print('Total : '+str(total_token))

Correct Token:  30157
False Token:  109907
Total : 140064


In [7]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")
def preprocess_function(examples):
    return tokenizer(examples["text"], padding=True, max_length = 20, truncation=True)

In [8]:
training_size = int(total_token*0.80)
# import io
# train_file = open('train_file_data',mode='w', encoding='utf-8')
# test_file = open('test_file_data','w')
# train_text_data = text_data[:training_size]
# test_text_data = text_data[training_size:]

# train_file.write('\n'.join(train_text_data))
# test_file.write('\n'.join(test_text_data))


1939252

In [10]:

train_text_list = []
train_label_list = []
test_text_list = []
test_label_list = []
for index in range(0,training_size):
    train_text_list.append(generate_dataset[index]['text'])
    train_label_list.append(generate_dataset[index]['label'])
for index in range(training_size,len(generate_dataset)):
    test_text_list.append(generate_dataset[index]['text'])
    test_label_list.append(generate_dataset[index]['label'])



In [149]:
# from datasets import DatasetDict, Dataset
# train_df = pd.DataFrame({
#      "text" : train_text_list,
#      "label" : train_label_list
# })

# test_df = pd.DataFrame({
#      "text" : test_text_list,
#      "label" : test_label_list
# })
# train_dataset = Dataset.from_dict(train_df)
# test_dataset = Dataset.from_dict(test_df)
# my_dataset_dict = DatasetDict({"train":train_dataset,"test":test_dataset})

In [150]:
# tokenized_my_dataset_dict = my_dataset_dict.map(preprocess_function, batched=True)
# # tokenized_my_dataset_dict = tokenized_my_dataset_dict.remove_columns(
# #     my_dataset_dict["train"].column_names
# # )

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [26]:
# from transformers import DataCollatorWithPadding

# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

Training data

In [107]:
# from pathlib import Path
# def read_imdb_split(split_dir):
#     split_dir = Path(split_dir)
#     texts = []
#     labels = []
#     for label_dir in ["pos", "neg"]:
#         for text_file in (split_dir/label_dir).iterdir():
#             texts.append(text_file.read_text())
#             labels.append(0 if label_dir is "neg" else 1)
#     return texts, labels

# train_texts, train_labels = read_imdb_split('aclImdb/train')
# test_texts, test_labels = read_imdb_split('aclImdb/test')

In [12]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_text_list, train_label_list, test_size=.2)

In [14]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(test_text_list, truncation=True, padding=True)

In [15]:
import torch

class ProtoQADataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = ProtoQADataset(train_encodings, train_labels)
val_dataset = ProtoQADataset(val_encodings, val_labels)
test_dataset = ProtoQADataset(test_encodings, test_label_list)


In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset             # evaluation dataset
)

trainer.train()

In [43]:
test_preds = trainer.predict(test_dataset).predictions
print(test_preds)
print((test_preds[:,0] > 0).sum())

***** Running Prediction *****
  Num examples = 28013
  Batch size = 64
/nas/home/skadvani/anaconda3/envs/protoqa/lib/python3.6/site-packages/torch/nn/parallel/_functions.py:61: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


[[-2.105601   1.8828943]
 [-2.2609258  2.0012233]
 [-1.8072803  1.5793715]
 ...
 [-1.192181   1.0203533]
 [-1.8413154  1.5331643]
 [-1.9574525  1.5936884]]
1


In [42]:
sum = 0 
for label in test_dataset.labels:
    if label == 0: sum+=1
print(sum/len(test_dataset.labels))

sum = 0 
for label in train_dataset.labels:
    if label == 0: sum+=1
print(sum/len(train_dataset.labels))

0.10780708956555884
0.10294511378848728


In [ ]:
# from torch.utils.data import DataLoader
# from transformers import DistilBertForSequenceClassification, AdamW

# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
# model.to(device)
# model.train()

# train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)

# optim = AdamW(model.parameters(), lr=5e-5)

# for epoch in range(3):
#     print('epoch ', epoch)
#     for batch in train_loader:
#         optim.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['labels'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs[0]
#         loss.backward()
#         optim.step()

# model.eval()